In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [60]:
import numpy as np
import pandas as pd
import sklearn
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from sklearn import feature_extraction,model_selection
from sklearn.metrics import confusion_matrix
import glob
import os
import string
import tensorflow as tf
import keras as K
import zipfile
from itertools import chain
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix, classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
# importing Word2Vec
from gensim.models import Word2Vec

In [0]:
zip_ref = zipfile.ZipFile("/content/drive/My Drive/hate-speech-dataset-master.zip", 'r')
zip_ref.extractall()
zip_ref.close()
file_path="/content/hate-speech-dataset-master/all_files"

In [63]:
read_files=glob.glob(os.path.join(file_path,"*.txt"))
data=[]
file_name=[]
for files in read_files:
    with open(files,encoding='utf-8') as f:
        data.append(f.read())
        file_name.append(os.path.splitext(os.path.basename(files))[0])
data=pd.DataFrame(data,index=file_name)
data.head()

0
13639258_1  I just cannot understand someone wanting to ri...
30680004_3  What educational goals or philosophies influen...
13504039_3  Los Angeles County sheriff 's officials arrest...
13465908_1  France has been gone for a long time , we will...
14665319_1                                     84% French ...

In [64]:
Output=pd.read_csv("/content/hate-speech-dataset-master/annotations_metadata.csv",index_col=0)
Output=Output.drop(['user_id','subforum_id','num_contexts'],axis=1).sort_index()
Output.head()

label
file_id            
12834217_1   noHate
12834217_10  noHate
12834217_2   noHate
12834217_3   noHate
12834217_4     hate

In [65]:
dataSet=pd.merge(data,Output,how='outer',left_index=True,right_index=True)
dataSet=dataSet[(dataSet['label']=="noHate")|(dataSet['label']=='hate')]
dataSet.rename(columns={0:'Texts'},inplace=True)
dataSet['label']=(dataSet['label']=='hate').astype(int)
dataSet.head()

Texts  label
12834217_1   As of March 13th , 2014 , the booklet had been...      0
12834217_10  Thank you in advance. : ) Download the youtube...      0
12834217_2   In order to help increase the booklets downloa...      0
12834217_3   ( Simply copy and paste the following text int...      0
12834217_4   Click below for a FREE download of a colorfull...      1

In [66]:
x = dataSet['Texts']
y = dataSet['label']
print(x.head(), y.head())

12834217_1     As of March 13th , 2014 , the booklet had been...
12834217_10    Thank you in advance. : ) Download the youtube...
12834217_2     In order to help increase the booklets downloa...
12834217_3     ( Simply copy and paste the following text int...
12834217_4     Click below for a FREE download of a colorfull...
Name: Texts, dtype: object 12834217_1     0
12834217_10    0
12834217_2     0
12834217_3     0
12834217_4     1
Name: label, dtype: int64


In [67]:
from keras.utils.np_utils import to_categorical
y = to_categorical(np.asarray(y))
print(x)

12834217_1     As of March 13th , 2014 , the booklet had been...
12834217_10    Thank you in advance. : ) Download the youtube...
12834217_2     In order to help increase the booklets downloa...
12834217_3     ( Simply copy and paste the following text int...
12834217_4     Click below for a FREE download of a colorfull...
                                     ...                        
33677015_1     Apparently he came to the conclusion that his ...
33677019_1     Wish we at least had a Marine Le Pen to vote f...
33677019_2     Its like the choices are white genocide candid...
33677053_1     Why White people used to say that sex was a si...
33677053_2                                        Now I get it !
Name: Texts, Length: 10703, dtype: object


In [0]:
tk=Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=" ")
tk.fit_on_texts(x)
word_index=tk.word_index
#print(index)
x2 = tk.texts_to_sequences(x)

In [69]:
embedding_vecor_length =146
padded_docs = pad_sequences(x2, maxlen=embedding_vecor_length, padding='post')
print (padded_docs)

[[ 26   6 515 ...   0   0   0]
 [200  11   7 ...   0   0   0]
 [  7 701   2 ...   0   0   0]
 ...
 [111  28   1 ...   0   0   0]
 [148  18  43 ...   0   0   0]
 [100   3  53 ...   0   0   0]]


In [70]:
vocab_size = len(tk.word_index)
print(vocab_size)

16002


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_docs, y, test_size=0.30, random_state=42)

In [0]:
# Creating embedding matrix
# Using glove.twitter.6B.50d.txt

In [73]:
#load pre-trained embedding:
embeddings_index = {}
f = open('/content/drive/My Drive/glove.6B.50d.txt', encoding="utf8")
for line in f:
    values = line.split()#This splits each values in a  line with spaces
    word = values[0] # This store the first values which is the word in english
    coefs = np.asarray(values[1:], dtype='float32') # This converts all splitted values from 1 to last(expect first word) into an array
    embeddings_index[word] = coefs # english word = array of numerical values
f.close()
print('Total %s word vectors in Glove 6B 50d.' % len(embeddings_index))

Total 400000 word vectors in Glove 6B 100d.


In [0]:
# Initialize the weight for each word for the embedding layer. Words not present
# in the dictionary initialized by random value 
EMBEDDING_DIM = 50
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
#embedding_matrix = (16433, 100) NOTE: It is from uniques words in our tweets
for word, i in word_index.items(): # Selecting word from our tweet unique words list for changing with the values from glove
    embedding_vector = embeddings_index.get(word) # Finding our tweet words in glove and taking its array of 100 values
    if embedding_vector is not None: 
#words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [75]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Embedding
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D
from keras.utils.vis_utils import plot_model
model = Sequential()
model.add(Embedding(input_dim = vocab_size + 1, output_dim = 50, weights=[embedding_matrix], input_length=embedding_vecor_length))
model.add(Conv1D(filters = 128,kernel_size=3, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Flatten())
model.add(Dense(256, activation = 'relu'))
model.add(Dense(2, activation = 'softmax'))
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 146, 50)           800150    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 144, 128)          19328     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 28, 128)           0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 3584)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               917760    
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 514       
Total params: 1,737,752
Trainable params: 1,737,752
Non-trainable params: 0
____________________________________________

In [76]:
model.fit(X_train, y_train, validation_data = (X_test, y_test),epochs=100,batch_size=100, verbose=2)

Train on 7492 samples, validate on 3211 samples
Epoch 1/100
 - 1s - loss: 0.4464 - acc: 0.8783 - val_loss: 0.3312 - val_acc: 0.8851
Epoch 2/100
 - 1s - loss: 0.2786 - acc: 0.8940 - val_loss: 0.2777 - val_acc: 0.8904
Epoch 3/100
 - 1s - loss: 0.2288 - acc: 0.9083 - val_loss: 0.2642 - val_acc: 0.8913
Epoch 4/100
 - 1s - loss: 0.1794 - acc: 0.9285 - val_loss: 0.2636 - val_acc: 0.8926
Epoch 5/100
 - 1s - loss: 0.1223 - acc: 0.9556 - val_loss: 0.2822 - val_acc: 0.8929
Epoch 6/100
 - 1s - loss: 0.0754 - acc: 0.9746 - val_loss: 0.3244 - val_acc: 0.8969
Epoch 7/100
 - 1s - loss: 0.0364 - acc: 0.9905 - val_loss: 0.3641 - val_acc: 0.8866
Epoch 8/100
 - 1s - loss: 0.0170 - acc: 0.9971 - val_loss: 0.4446 - val_acc: 0.8960
Epoch 9/100
 - 1s - loss: 0.0085 - acc: 0.9989 - val_loss: 0.4890 - val_acc: 0.8988
Epoch 10/100
 - 1s - loss: 0.0041 - acc: 0.9996 - val_loss: 0.5072 - val_acc: 0.8954
Epoch 11/100
 - 1s - loss: 0.0025 - acc: 0.9999 - val_loss: 0.5598 - val_acc: 0.8982
Epoch 12/100
 - 1s - loss:

In [77]:
predictions_test = model.predict(X_test)
print(predictions_test)

[[1.0000000e+00 4.1453282e-11]
 [1.0000000e+00 4.3182489e-20]
 [1.0000000e+00 6.6551864e-24]
 ...
 [1.0000000e+00 1.3987560e-14]
 [1.0000000e+00 4.5995466e-10]
 [1.0000000e+00 7.5186493e-13]]


In [0]:
predictions_test1 = np.zeros_like(predictions_test)
predictions_test1[np.arange(len(predictions_test)), predictions_test.argmax(1)] = 1

In [79]:
print(classification_report(y_test,predictions_test1))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      2842
           1       0.55      0.39      0.46       369

   micro avg       0.89      0.89      0.89      3211
   macro avg       0.74      0.67      0.70      3211
weighted avg       0.88      0.89      0.89      3211
 samples avg       0.89      0.89      0.89      3211

